In [ ]:
import feedparser
import dateutil.parser
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import mongo_utils
from datetime import datetime as dt
import json
import re


In [ ]:
dbname = mongo_utils.get_database()

In [ ]:
urls = ['https://www.investor.bg/rss/latest',
    ]
feeds = [feedparser.parse(url)['entries'] for url in urls]
feed = [item for feed in feeds for item in feed]

In [ ]:
print(feed)


In [ ]:
for newslink in feed:
  try:
    #print(newslink["id"])
    nlink=newslink["link"]
    print(nlink)
    collection_name = dbname["investor_scrape_1"]
    if list(collection_name.find({"news_link" : nlink })):
        print("Article Exists")
    else:
        dt_obj=newslink["published"]
        date_posted = dt.strptime(dt_obj, '%a, %d %b %Y %H:%M:%S %z').timestamp()
        print(date_posted)
        news_title=newslink["title"]
        print(news_title)
        page = requests.get(nlink)
        data = page.text
        soup = BeautifulSoup(data, 'html.parser')
        #print(soup)
        #text = soup.find_all('p')
        artcl = soup.find("script", type="application/ld+json").text
        artcl = artcl.replace('  ','').replace('\n',' ').replace('\xa0', ' ').replace('\r', '').replace('\t', '')    
        artcl = json.loads(artcl)
        text=(artcl["articleBody"])
        news_payload = {
        "date_posted" : date_posted,
        "news_link" : nlink,
        "news_text" : text,
        "news_title" : news_title
        }
        collection_name.insert_many([news_payload])
        print(news_payload)
  except:
    pass
